In [ ]:
import numpy as np
import cv2
import os
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Activation, Conv2DTranspose, Concatenate, Input, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow .keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
import logging
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score
import csv

In [ ]:
dataset_path = "../content/drive/MyDrive/U-Net"
files_directory = os.path.join("files_UNET","U-Net")
model_file = os.path.join(files_directory, "model-U-Net.h5")
log_file = os.path.join(files_directory, "log.csv")

In [ ]:
def create_direct(path):
  if not os.path.exists(path):
    os.makedirs(path)
create_direct(files_directory)

In [ ]:
#Convolution block
def con_block(inp, number_filters):
  x = Conv2D(number_filters, 3, padding = "same") (inp)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Conv2D(number_filters, 3, padding = "same") (x)
  x = BatchNormalization()(x)
  X = Activation("relu")(x)
  return x

#Encoder block
def enc_block(inp, number_filters):
  x = con_block(inp, number_filters)
  p = MaxPool2D((2,2))(x)
  return x,p

#Decoder block
def dec_block(inp, skip, number_filters):
  x = Conv2DTranspose(number_filters, (2,2), strides=2, padding="same")(inp)
  x = Concatenate() ([x,skip])
  x = con_block(x, number_filters)
  return x

In [ ]:
#Buidling architecture
def build_unet(inp_shape):
  inp = Input(inp_shape)
  #Encoder
  skip1, enc1 = enc_block(inp, 64)
  skip2, enc2 = enc_block(enc1, 128)
  skip3, enc3 = enc_block(enc2, 256)
  skip4, enc4 = enc_block(enc3, 512)

    #Bridge
  b1 = con_block(enc4, 1024)

  #Decoder
  dec1 = dec_block(b1, skip4, 512)
  dec2 = dec_block(dec1, skip3, 256)
  dec3 = dec_block(dec2, skip2, 128)
  dec4 = dec_block(dec3, skip1, 64)

  output = Conv2D(1, 1, padding="same", activation="sigmoid")(dec4)
  model = Model(inp, output, name="U-net")
  return model

In [ ]:
# Seed the environment
np.random.seed(45)
tf.random.set_seed(45)
 # Hyperparameters
batch_size = 4
lr = 1e-6
epochs = 5
height=640
weight=1280

In [ ]:
#Loading training and validation
def load_dataset(path):
  train_x = sorted(glob(os.path.join(path, "train", "images", "*")))
  train_y = sorted(glob(os.path.join(path, "train", "masks", "*")))
  val_x = sorted(glob(os.path.join(path, "val", "images", "*")))
  val_y = sorted(glob(os.path.join(path, "val", "masks", "*")))
  return (train_x,train_y), (val_x,val_y)

In [ ]:
#Reading images and masks
def read_image(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = x/255.0
  return x
def read_mask(path):
  path = path.decode()
  x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  x = x/255.0
  x = np.expand_dims(x, axis=-1)
  return x

In [ ]:
#Data preprocessing
def tf_parse(x, y):
  def _parse(x,y):
    x = read_image(x)
    y = read_mask(y)
    x = tf.image.resize(x, (640, 1280))
    y = tf.image.resize(y, (640, 1280))
    x = tf.dtypes.cast(x, tf.float16)
    y = tf.dtypes.cast(y, tf.float16)
    return x, y

  x, y = tf.numpy_function(_parse, [x, y], [tf.float16, tf.float16])
  x.set_shape([640, 1280, 3])
  y.set_shape([640, 1280, 1])
  return x, y


In [ ]:
def tf_dataset(x, y, batch=4):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(batch)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

In [ ]:
#Splitting
(train_x, train_y), (val_x, val_y) = load_dataset(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Val: {len(val_x)} - {len(val_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
val_dataset = tf_dataset(val_x, val_y, batch=batch_size)

for x, y in val_dataset:
  print(x.shape,y.shape)

Train: 500 - 500
Val: 50 - 50
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(4, 640, 1280, 3) (4, 640, 1280, 1)
(2, 640, 1280, 3) (2, 640, 1280, 1)


In [ ]:
inp_shape = (height,weight,3)
model = build_unet(inp_shape)
model.summary()

Model: "U-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 640, 1280,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 640, 1280, 6  1792        ['input_2[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization_18 (BatchN  (None, 640, 1280, 6  256        ['conv2d_19[0][0]']              
 ormalization)                  4)                                                            

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=False),
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience = 4),
    CSVLogger(log_file),
       ]

In [ ]:
class MetricsCallback(Callback):
    def __init__(self, val_data):
        super().__init__()
        self.val_data = val_data

        # Initialize logging
        logging.basicConfig(filename='log.csv', filemode='a', format='%(asctime)s, %(message)s', level=logging.INFO)

    def on_epoch_end(self, epoch, logs=None):
        val_predictions = self.model.predict(self.val_data[0])
        val_true_labels = self.val_data[1]

        # Calculate metrics
        accuracy = accuracy_score(np.argmax(val_true_labels, axis=1), np.argmax(val_predictions, axis=1))
        precision = precision_score(np.argmax(val_true_labels, axis=1), np.argmax(val_predictions, axis=1))
        recall = recall_score(np.argmax(val_true_labels, axis=1), np.argmax(val_predictions, axis=1))
        f1 = f1_score(np.argmax(val_true_labels, axis=1), np.argmax(val_predictions, axis=1))
        mse = mean_squared_error(val_true_labels, val_predictions)
        rmse = np.sqrt(mse)
        r2 = r2_score(val_true_labels, val_predictions)

        # Log metrics
        logging.info(f'Epoch: {epoch}, Loss: {logs["loss"]}, Val_Loss: {logs["val_loss"]}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R-Squared: {r2:.4f}')


In [ ]:
# Configure the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=2)]
)
# Training
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/5
125/125 [==============================] - ETA: 0s - loss: 0.4069 - accuracy: 0.8481 - mean_io_u: 0.4488 
Epoch 1: val_loss improved from inf to 4062.67114, saving model to files_UNET/U-Net/model-U-Net.h5
125/125 [==============================] - 4818s 38s/step - loss: 0.4069 - accuracy: 0.8481 - mean_io_u: 0.4488 - val_loss: 4062.6711 - val_accuracy: 0.2061 - val_mean_io_u: 0.1031 - lr: 0.0010
Epoch 2/5
125/125 [==============================] - ETA: 0s - loss: 0.2429 - accuracy: 0.9104 - mean_io_u: 0.4852 
Epoch 2: val_loss improved from 4062.67114 to 227.71333, saving model to files_UNET/U-Net/model-U-Net.h5
125/125 [==============================] - 4746s 38s/step - loss: 0.2429 - accuracy: 0.9104 - mean_io_u: 0.4852 - val_loss: 227.7133 - val_accuracy: 0.2064 - val_mean_io_u: 0.1107 - lr: 0.0010
Epoch 3/5
125/125 [==============================] - ETA: 0s - loss: 0.1893 - accuracy: 0.9263 - mean_io_u: 0.5003 
Epoch 3: val_loss improved from 227.71333 to 11.01087, saving

In [ ]:
import os
import time
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

In [ ]:
# Seed the environment
np.random.seed(45)
tf.random.set_seed(45)
 # Hyperparameters
batch_size = 4
lr = 1e-6
epochs = 5
height=640
weight=1280

In [ ]:
  data_path="../content/drive/MyDrive/U-Net"
  save_path = os.path.join(data_path, "pred")
  model_path = '../content/drive/MyDrive/U-Net/files_U-Net/model-U-Net.h5'

In [ ]:
def create_dir(path):
  if not os.path.exists(path):
    os.makedirs(path)


In [ ]:
create_dir(save_path)

In [ ]:
#Load the model
model = tf.keras.models.load_model(model_path)

model.summary()

Model: "U-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 1280,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 640, 1280, 6  1792        ['input_1[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 640, 1280, 6  256        ['conv2d[0][0]']                 
 alization)                     4)                                                            

In [ ]:
import os
from glob import glob

data_path = "../content/drive/MyDrive/U-Net/test/images"

# Construct the file pattern
test_pattern = os.path.join(data_path, "*")

# Load the test images
test_x = sorted(glob(test_pattern))

# Print the number of test images
print(f"Test Images: {len(test_x)}")


Test Images: 50


In [ ]:
time_taken = []

for x in tqdm(test_x):
  name = x.split("/") [-1]
  x = cv2.imread(x, cv2.IMREAD_COLOR)
  x = x / 255.0
  x = np.expand_dims(x, axis=0)

  start_time = time.time()
  p = model.predict(x)
  total_time = time.time() - start_time
  time_taken.append(total_time)
  p = p > 0.5
  p = p * 255
  cv2.imwrite(os.path.join(save_path, name), p)

  break

  0%|          | 0/50 [00:00<?, ?it/s]

1/1 [==============================] - 3s 3s/step


  0%|          | 0/50 [00:02<?, ?it/s]


In [ ]:
import os

if not os.path.exists("../content/drive/MyDrive/U-Net/pred"):
  os.makedirs("../content/drive/MyDrive/U-Net/pred")


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

# Load the U-Net model.
model = load_model(model_path)

# Get the list of paths to the test images.
test_image_paths = os.listdir("../content/drive/MyDrive/U-Net/test/images")

# Define the save_path directory.
save_path = os.path.join(data_path, "pred")

# Ensure the save_path directory exists.
os.makedirs(save_path, exist_ok=True)

# Iterate over the test images.
with tqdm(total=len(test_image_paths)) as pbar:
    for image_path in test_image_paths:

        # Read the original image.
        original_image = cv2.imread(os.path.join("../content/drive/MyDrive/U-Net/test/images", image_path), cv2.IMREAD_COLOR)

        # Preprocess the image for prediction.
        image = original_image / 255.0
        image = np.expand_dims(image, axis=0)

        # Predict the mask.
        mask = model.predict(image)

        # Threshold the predicted mask (assuming binary segmentation).
        threshold = 0.5
        mask = (mask > threshold).astype(np.uint8) * 255

        # Save the predicted mask to the save_path directory.
        tqdm.write(f"Saving predicted mask for image: {image_path}")
        cv2.imwrite(os.path.join(save_path, image_path.split("/")[-1]), mask)

        # Update the progress bar.
        pbar.update(1)


  0%|          | 0/50 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  2%|▏         | 1/50 [00:02<02:00,  2.46s/it]

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_22949014.JPG
1/1 [==============================] - 2s 2s/step


  4%|▍         | 2/50 [00:04<01:46,  2.21s/it]

...

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_23004364.JPG
1/1 [==============================] - 2s 2s/step


 98%|█████████▊| 49/50 [01:42<00:02,  2.06s/it]

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_23008400.JPG
1/1 [==============================] - 2s 2s/step


100%|██████████| 50/50 [01:44<00:00,  2.09s/it]

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_23033914.JPG


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

# Load the U-Net model.
model = load_model(model_path)

# Get the list of paths to the test images.
test_image_paths = os.listdir("../content/drive/MyDrive/U-Net/test/images")

# Define the save_path directory.
save_path = os.path.join(data_path, "pred")

# Ensure the save_path directory exists.
os.makedirs(save_path, exist_ok=True)

# Iterate over the test images.
with tqdm(total=len(test_image_paths)) as pbar:
    for image_path in test_image_paths:

        # Read the original image.
        original_image = cv2.imread(os.path.join("../content/drive/MyDrive/U-Net/test/images", image_path), cv2.IMREAD_COLOR)

        # Preprocess the image for prediction.
        image = original_image / 255.0
        image = np.expand_dims(image, axis=0)

        # Predict the mask.
        mask = model.predict(image)

        # Threshold the predicted mask (assuming binary segmentation).
        threshold = 0.5
        mask = (mask > threshold).astype(np.uint8) * 255

        # Save the predicted mask to the save_path directory.
        tqdm.write(f"Saving predicted mask for image: {image_path}")
        cv2.imwrite(os.path.join(save_path, image_path.split("/")[-1]), mask[..., 0])

        # Update the progress bar.
        pbar.update(1)


  0%|          | 0/50 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  2%|▏         | 1/50 [00:02<02:04,  2.54s/it]

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_22949014.JPG
1/1 [==============================] - 2s 2s/step


  4%|▍         | 2/50 [00:04<01:49,  2.29s/it]

...

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_23004364.JPG
1/1 [==============================] - 2s 2s/step


 98%|█████████▊| 49/50 [01:40<00:02,  2.03s/it]

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_23008400.JPG
1/1 [==============================] - 2s 2s/step


100%|██████████| 50/50 [01:43<00:00,  2.06s/it]

Saving predicted mask for image: DMI_OS_UPPER_REFLECTED_IR_23033914.JPG


In [ ]:
from google.colab.patches import cv2_imshow


In [ ]:
import cv2
import numpy as np
import os

# Define the paths to the original images, ground truth masks, and predicted masks.
original_image_path = "../content/drive/MyDrive/U-Net/test/images"
ground_truth_mask_path = "../content/drive/MyDrive/U-Net/test/masks"
predicted_mask_path = "../content/drive/MyDrive/U-Net/pred"

# Get the list of paths to the original images.
original_image_paths = os.listdir(original_image_path)

# Iterate over the original images.
for image_path in original_image_paths:

    # Read the original image.
    original_image = cv2.imread(os.path.join(original_image_path, image_path), cv2.IMREAD_COLOR)

    # Read the ground truth mask.
    ground_truth_mask = cv2.imread(os.path.join(ground_truth_mask_path, image_path), cv2.IMREAD_GRAYSCALE)

    # Read the predicted mask.
    predicted_mask = cv2.imread(os.path.join(predicted_mask_path, image_path), cv2.IMREAD_GRAYSCALE)

    # Display the original image, ground truth mask, and predicted mask.
    cv2.imshow("Original image", original_image_path)
    cv2.imshow("Ground truth mask", ground_truth_mask_path)
    cv2.imshow("Predicted mask", predicted_mask_path)

    # Wait for key press.
    cv2.waitKey(0)

# Close all windows.
cv2.destroyAllWindows()


DisabledFunctionError: ignored